In [ ]:
!pip install transformers datasets faiss-cpu -q
!pip install google-api-python-client
!pip install scikit-learn
!pip install requests

In [ ]:
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from lime.lime_text import LimeTextExplainer
from sklearn.pipeline import make_pipeline


model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)


def predict_fn(texts):
    encodings = tokenizer(texts, truncation=True, padding=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**encodings)
    return torch.nn.functional.softmax(outputs.logits, dim=-1).numpy()

explainer = LimeTextExplainer(class_names=['Negative', 'Positive'])  # Ajuste as classes conforme necessário

def explain_instance(text, model, num_features=10):
    explanation = explainer.explain_instance(
        text,
        predict_fn,
        num_features=num_features
    )
    return explanation

In [ ]:
#Example
text = "I love this product! It works great and I'm very happy with it."

explanation = explain_instance(text, model)
explanation.show_in_notebook()
print("Top features influencing the model's decision:")
for feature, score in explanation.as_list():
    print(f"Feature: {feature}, Score: {score}")